In [19]:
#importing necessary libraries
import cv2
import openai
import pytesseract
import pyttsx3
import numpy as np

In [20]:
#setting openai key
openai.api_key="use your openAI key here"

In [21]:
#setting pytesseract path
pytesseract.pytesseract.tesseract_cmd=r'C:\Users\aswat\AppData\Local\Tesseract-OCR\tesseract.exe'

In [22]:
#loading image
img = cv2.imread(r"C:\Users\aswat\Desktop\DL PROJECTS\opencvproject\fre.jpg")
img

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [23]:
if img is not None:
    img2 = cv2.resize(img, (800, 750))

    # # Display the resized image
    # cv2.imshow("Resized Image", img2)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # converting img to gray scale
    gray_img=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    cv2.imshow('image',gray_img)
    cv2.waitKey()
    cv2.destroyAllWindows()

    #using tesseract to extract text from img
    extracted_text=pytesseract.image_to_string(gray_img)

    # listing the lang options
    lang_options=['en','fr','es','de','hi','ml','kr']
    
    # user input for source lang
    print("select the source language")
    for i,lang in enumerate(lang_options,1):
        print(i,lang)
    source_lang=int(input("enter source language choice num"))

    #user input for target lang
    print("select the target language")
    for i,lang in enumerate(lang_options,1):
        print(i,lang)
    target_lang=int(input("enter the num for target language choice"))

    # mapping user i/p for both to crspndng lang optn
    source_language=lang_options[(source_lang) -1]
    target_language=lang_options[(target_lang) -1]

    
    # Constructing prompt 
    full_prompt = f"Translate the following {source_language} passage to {target_language}:\n{extracted_text}"

    # Translating extracted lang to target language
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=full_prompt,
        max_tokens=3500,
        temperature=0.7,    #0.5, (closer to 1 creative output)(closer to 0 deterministic output)
        top_p=0.9           #0.4  (closer to 1 broader selection)(closer to 0 narrow selection)

    )
    #extracting the translated text from api response
    translated_txt=response['choices'][0]['text'].strip()

    #saving the org txt and translated txt in a file
    with open('translated_output.txt','w',encoding='utf-8') as file:
        file.write(f"Original Text ({source_language}):\n{extracted_text}\n\n")
        file.write(f"translated txt({target_language}):\n{translated_txt}")
    print("output saved to translated_output.txt")

    # Creating a larger blank image to insert the extracted text
    image = np.zeros((7000, 7000, 3), dtype=np.uint8) 

    text_position = (8, 40)  

    #NeW Line char
    newline = '\n'
    translated_txt_multiline = f"Translated text ({target_language}):{newline}{translated_txt.replace(',', newline)}"

    # Add the modified translated text to the image
    lines = translated_txt_multiline.split('\n')

    # Use enumerate to draw each line
    for i, line in enumerate(lines):
        y = text_position[1] + i * 30
        line = line.strip()  # Strip last white spaces
        cv2.putText(image, line, (text_position[0], y), cv2.FONT_HERSHEY_COMPLEX, 0.4, (255,255,255), 1)

    # Display the image
    cv2.imshow('translated', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("error")
    


select the source language
1 en
2 fr
3 es
4 de
5 hi
6 ml
7 kr
select the target language
1 en
2 fr
3 es
4 de
5 hi
6 ml
7 kr
output saved to translated_output.txt


In [24]:
# Text-to-speech
txt_sp = pyttsx3.init()
txt_sp.say("Translation successful. Here is the translation.")
txt_sp.say(translated_txt)
txt_sp.runAndWait()